In [30]:
def get_color(acc2info):
    phyla2color = {p:c for p,c in zip(sorted(set(acc2info.values())),
                                      px.colors.qualitative.T10 + px.colors.qualitative.Alphabet)}
    phyla2color['Proteobacteria']='#3C6A54'
    phyla2color['Alphaproteobacteria']='#FAD4A2'
    phyla2color['Betaproteobacteria']='#56CEFF'
    phyla2color['Gammaproteobacteria']='#98C995'
    phyla2color['Epsilonproteobacteria']='#52B6DC'
    phyla2color['Deltaproteobacteria']='#DAE8AA'
    phyla2color['Oligoflexia']='#FF8028'
    phyla2color['Candidatus Muproteobacteria'] = '#BD2B1C'
    phyla2color['Zetaproteobacteria'] = '#63680c'
    phyla2color['Actinobacteria']= '#E15F99'
    phyla2color['Bacteroidetes'] = '#DA16FF'
    phyla2color['Chloroflexi'] = '#511CFB'
    phyla2color['Cyanobacteria'] = '#00A08B'
    phyla2color['Euryarchaeota'] = '#FC0080'
    phyla2color['Firmicutes'] = '#B2828D'
    phyla2color['Gemmatimonadetes'] ='#6C7C32'
    phyla2color['Nitrospinae'] = '#862A16'
    phyla2color['Nitrospirae'] = '#A777F1'
    phyla2color['Planctomycetes'] = '#620042'
    phyla2color['Deinococcus-Thermus'] = '#B6C360'
    phyla2color['Acidithiobacillia'] = '#336666'
    phyla2color['Crenarchaeota']= '#BAB0AC'
    phyla2color['Deinococcus-Thermus']= '#3283FE'
    phyla2color['Halobacteria'] = '#3283FE'

    
    return phyla2color

In [353]:
def to_color_range(acc2info, info2color, pos='range'):

    acc2color = {acc: info2color[phyla] for acc, phyla in acc2info.items()}
    if pos == 'range':
        anno_text = '\n'.join(['%s,%s,%s,%s' % (acc, pos, color, acc2info[acc])
                              for acc,color in acc2color.items()])
    else:
        anno_text = '\n'.join(['%s,%s,%s,normal,3' % (acc, pos, color)
                              for acc,color in acc2color.items()])
    
    os.chdir('/share/home-user/hyc/db/')
    template_text = open('./itol_template/colors_styles_template.txt').read()

    # 形成最后输出文件
    out_text = template_text + '\n' + anno_text

    return out_text

In [59]:
def to_color_style(acc2info,info2color,dataset,pos,bg=False):
    # 该部分可封装
    # 获取访问码与颜色的对应
    acc2color = {acc: info2color[phyla] for acc, phyla in acc2info.items()}

    # data 注释主体输出
    if not bg:
        # data 注释主体输出
        annote_text = '\n'.join(['%s,%s,%s,1,normal' % (acc, pos, color)  
                                 for acc, color in acc2color.items()])
    else:
        annote_text = '\n'.join(['%s,%s,#ffffff,1,normal,%s' 
                                 % (acc, pos, color) 
                                 for acc, color in acc2color.items()])
    
    
    owned_info2color = {info:info2color[info] for acc,info in acc2info.items()}

    # legend 形成，以及整个dataset的名字
    legend_text = std_legend(dataset, owned_info2color)
    dataset_label = dataset

    # 打开模板文件
    os.chdir('/share/home-user/hyc/db/')
    template_text = open('./itol_template/dataset_styles_template.txt').read()

    # 输入到预先修改为format格式的文本文件
    template_text = template_text.format(legend_text=legend_text,
                                         dataset_label=dataset_label)

    # 形成最后输出文件
    out_text = template_text + '\n' + annote_text

    return out_text

In [257]:
def to_external_label(acc2info,dataset='label info'):
    os.chdir('/share/home-user/hyc/db/')
    template_text = open('./itol_template/dataset_text_template.txt').read()
    template_text = template_text.format(dataset_label = dataset)
    
    annotate_text = '\n'.join(['%s,%s,1,#000000,normal,1' %(k,v) 
                               for k,v in acc2info.items()])
    
    return template_text + '\n' + annotate_text

In [255]:
def label_text(acc2info):
    os.chdir('/share/home-user/hyc/db/')
    template_text = open('./itol_template/labels_template.txt').read()
    
    annotate_text = '\n'.join(['%s,%s' %(k,v) 
                               for k,v in acc2info.items()])
    
    return template_text + '\n' + annotate_text

In [81]:
def to_popup(acc2info):
    os.chdir('/share/home-user/hyc/db/')
    template_text = open('./itol_template/popup_info_template.txt').read()
    
    annotate_text = '\n'.join(['%s,accession,%s' %(k,k) 
                               for k,v in acc2info.items()])
    
    return template_text + '\n' + annotate_text

# above are annotate part
# below are reference annotate

In [2]:
from mytool.customise_annote_file import *
import os 
import pandas as pd
from ete3 import Tree
import re

from Bio import SeqIO

In [371]:
# annotate ref tree
# os.chdir('/share/home-user/hyc/work/5_tree/iqtree/ref_nr97/')
os.chdir('/share/home-user/hyc/work/5_tree/fasttree/ref_tree/')
gene = 'nirS'
# filepath = '%s/%s.contree' %(gene,gene)
filepath = '%s_ref' % gene

t = Tree(filepath, format=1)

tree_acc = t.get_leaf_names()

df = pd.read_excel('../../total_ref_outgroup_modified.xlsx',
                   usecols=['accession','phylum','len','tax'])
acc2phylum = dict(zip(df['accession'],df['phylum']))
phyla2color = get_color(acc2phylum)

df = df[df['accession'].isin(tree_acc)]

df = find_proteoclass(df)

acc2phylum = dict(zip(df['accession'],df['phylum']))
acc2phyclas = dict(zip(df['accession'],df['phy/class']))

In [60]:
os.chdir('/share/home-user/hyc/work/5_tree/')
gene = 'nirS'
ref_og_ids = [_.id for _ in SeqIO.parse('/share/home-user/hyc/work/3_cdhit/ref/%s_ref.nr97.fa'%gene,'fasta')]
# ref_ids = [_.id for _ in SeqIO.parse('/share/home-user/hyc/work/0_seq/ref/norB_ref.fasta','fasta')]
# og_ids = [_.id for _ in SeqIO.parse('/share/home-user/hyc/work/0_seq/outgroup/norB_outgroup.fasta','fasta')]
# ref_ids = [_.replace('[','_').replace(']','_') for _ in ref_ids]
# og_ids = [_.replace('[','_').replace(']','_') for _ in og_ids]

df = pd.read_excel('total_ref_outgroup.xlsx',
                   usecols=['accession','phylum','len','tax','gene','type'])

df = df[df['accession'].isin(ref_og_ids)]
df = find_proteoclass(df)

acc2phylum = dict(zip(df['accession'],df['phylum']))
acc2phyclas = dict(zip(df['accession'],df['phy/class']))

In [68]:
acc2phylum2 = {key.replace('[','_').replace(']','_'):acc2phylum[key] for key in acc2phylum.keys()}
acc2phyclas2 = {key.replace('[','_').replace(']','_'):acc2phyclas[key] for key in acc2phyclas.keys()}

In [61]:
out_text_phylum = to_color_strip(acc2phylum,phyla2color,'phylum')

acc2proteocla = {k:v for k,v in acc2phyclas.items() 
                 if v not in acc2phylum.values()}

out_text_proteoclass = to_color_strip(acc2proteocla,
                                      phyla2color,
                                      'Proteobacteria class')

os.chdir('/share/home-user/hyc/work/5_tree/annotate')
with open('./colorstrip/%s_ref_colorstrip_phylum.txt' 
          %(gene),'w') as f:
    f.write(out_text_phylum)

with open('./colorstrip/%s_ref_colorstrip_protepclass.txt' 
          %(gene), 'w') as f:
    f.write(out_text_proteoclass)

In [62]:
out_text_styles=to_color_style(acc2phyclas,phyla2color,'branch color','label,node',bg=True)

os.chdir('/share/home-user/hyc/work/5_tree/annotate')
with open('./colorstyle/%s_ref_colorstyle_phyclas.txt' 
          %(gene), 'w') as f:
    f.write(out_text_styles)

In [63]:
acc2len = dict(zip(df['accession'],df['len']))
# acc2len2 = {key.replace('[','_').replace(']','_'):acc2len[key] for key in acc2len.keys()}
out_text_bar = simplebar(acc2len,'%s reference sequence length' % gene)

os.chdir('/share/home-user/hyc/work/5_tree/annotate')
with open('./len/%s_ref_bar.txt' 
          %(gene), 'w') as f:
    f.write(out_text_bar)

In [64]:
# acc2gene = dict(zip(df['fa_name'],df['gene']))
# acc2gene2 = {key.replace('[','_').replace(']','_'):acc2gene[key] for key in acc2gene.keys()}
# gene2color = get_color(acc2gene2)
acc2type = dict(zip(df['accession'],df['type']))
type2color = {'ref':'#6BFF2C'}
# acc2type2 =  {key.replace('[','_').replace(']','_'):acc2type[key] for key in acc2type.keys()}

out_text_range = to_color_range(acc2type,type2color)
# out_text_range = to_color_range(acc2gene2,gene2color)
os.chdir('/share/home-user/hyc/work/5_tree/annotate')
with open('./colorrange/%s_ref_type_range.txt' %gene,'w') as f:
    f.write(out_text_range)

In [104]:
os.chdir('/share/home-user/hyc/work/5_tree/')
gene = 'nirK'
ref_ids = [_.id for _ in SeqIO.parse('/share/home-user/hyc/work/0_seq/ref/%s_ref.fasta'%gene,'fasta')]
og_ids = [_.id for _ in SeqIO.parse('/share/home-user/hyc/work/0_seq/outgroup/%s_outgroup.fasta'%gene,'fasta')]

# df = pd.read_excel('total_ref_outgroup.xlsx',
#                    usecols=['accession','type'])
# df = df[df['accession'].isin(ref_ids)]

acc2type = dict(zip(og_ids,['outgroup']*len(og_ids)))
type2color = {'outgroup':'#9400d3'}
# acc2type = dict(zip(df['accession'],df['type']))
# type2color = {'ref':'#6BFF2C','nirK outgroup':'#6BFF2C'}
out_text_range = to_color_range(acc2type,type2color)
os.chdir('/share/home-user/hyc/work/5_tree/annotate')
with open('./colorrange/%s_og_type_range.txt' %gene,'w') as f:
    f.write(out_text_range)

In [376]:
# to cluster size
os.chdir('/share/home-user/hyc/work/3_cdhit/ref/')

text = ''
acc2clstr=dict()

for line in open(gene+'_criteria.nr97.fasta.clstr'):
    if re.match('^>Cluster',line):
        if text:
            clstr = text.split('\n')[:-1]
            for _ in clstr:
                if '*' in _:
                    accid = _.split('>')[-1].split('...')[0]
                    clstr_no = len(clstr)
                    acc2clstr[accid]=clstr_no
                    break
            text = ''
    else:
        text += line

out_text_bar = simplebar(acc2clstr,'%s gene clstr' % gene)

os.chdir('/share/home-user/hyc/work/5_tree/annotate')
with open('./len/%s_ref_clstr.txt' 
          %(gene), 'w') as f:
    f.write(out_text_bar)

#########################################
#            annote gene tree           #
#########################################

In [17]:
# read main table, and do not change it
genome_meta = pd.read_csv('/share/home-user/hyc/db/my_Wol_2019/genome_10575_metadata.tsv',
                          sep = '\t',
                          header =0, usecols=['#genome','superkingdom','phylum','class'])

genome_dict = pd.read_csv('/share/home-user/hyc/db/my_Wol_2019/genome.dict',
                          sep = '\t',
                          header =0)

In [52]:
# annotate gene tree
os.chdir('/share/home-user/hyc/work/5_tree/iqtree/ref_10575/')
gene = 'nosZ'
filepath = '%s/%s.contree' %(gene,gene)
t = Tree(filepath, format=1)

tree_acc = t.get_leaf_names()
len(tree_acc)

752

In [35]:
gene = 'nosZ'

In [53]:
ref_og_ids = [_.id for _ in SeqIO.parse('/share/home-user/hyc/work/3_cdhit/ref/%s_ref.nr97.fa'%gene,'fasta')]
# ref_og_ids += [_.id for _ in SeqIO.parse('/share/home-user/hyc/work/0_seq/outgroup/norB_outgroup.fasta','fasta')]

# ref_og_ids = [_.replace('[','_').replace(']','_') for _ in ref_og_ids]
tree_acc = list(set(tree_acc) ^ set(ref_og_ids))
len(tree_acc)

617

In [37]:
df = pd.DataFrame(columns=['accession','phylum','class'])

df['accession'] = tree_acc

In [38]:
# change NC_9999.1_1111 to NC_9999.1 (NC_9999.1_1111:NC_9999.1)
tree_acc_t = {_:re.match(r'(\w+\.\d+)_\d+',_).group(1) for _ in tree_acc}

# get small table (G99999 NC_9999.1)
t_genome_dict = genome_dict[genome_dict['Accession'].isin(tree_acc_t.values())]
# change to dict for visit (NC_9999.1:G99999)
t_genome_dict = t_genome_dict.set_index('Accession').to_dict()['Genome']

# change to (NC_9999.1_1111:G99999)
tree_acc_t = {k:t_genome_dict[v] for k,v in tree_acc_t.items()}

In [46]:
# get small table (G99999 phylum class)
t_genome_meta = genome_meta[genome_meta['#genome'].isin(tree_acc_t.values())]

# phylum = [ t_genome_meta.loc[t_genome_meta['#genome']==v,'phylum'].values[0]
#           for k,v in tree_acc_t.items() ]

# tax_class = [ t_genome_meta.loc[t_genome_meta['#genome']==v,'class'].values[0] 
#              for k,v in tree_acc_t.items()]
phylum = []
tax_class = []
for k,v in tree_acc_t.items():
    phy = t_genome_meta.loc[t_genome_meta['#genome']==v,'phylum'].values[0]
    if str(phy) != 'nan':
        if phy.startswith('Candidatus'):
            tax_class.append('Candidatus')
            phylum.append('Candidatus')
        elif phy == "Proteobacteria":
            phylum.append(phy)
            tax_class.append(t_genome_meta.loc[t_genome_meta['#genome']==v,'class'].values[0])
        elif phy.endswith('archaeota'):
            phylum.append(phy)
            arch_clas = t_genome_meta.loc[t_genome_meta['#genome']==v,'class'].values[0]
            if str(arch_clas) != 'nan':
                tax_class.append(arch_clas)
            else:
                tax_class.append(phy)
        else:
            phylum.append(phy)
            tax_class.append(phy)
    else:
        kin = t_genome_meta.loc[t_genome_meta['#genome']==v,'superkingdom'].values[0]
        phylum.append(kin)
        tax_class.append(kin)
# list(set(tax_class).difference(set(phylum)))
df['phylum']=phylum
df['class']=tax_class

acc2phylum = dict(zip(df['accession'],df['phylum']))
acc2phyclas = dict(zip(df['accession'],df['class']))

In [48]:
phyla2color = get_color(acc2phylum)

In [49]:
phyclas2color = get_color(acc2phyclas)

In [50]:
phyclas2color

{'Acidobacteria': '#4C78A8',
 'Actinobacteria': '#E15F99',
 'Alphaproteobacteria': '#FAD4A2',
 'Archaea': '#72B7B2',
 'Bacteria': '#54A24B',
 'Bacteroidetes': '#DA16FF',
 'Balneolaeota': '#B279A2',
 'Betaproteobacteria': '#56CEFF',
 'Candidatus': '#9D755D',
 'Candidatus Muproteobacteria': '#BD2B1C',
 'Chlamydiae': '#AA0DFE',
 'Chlorobi': '#3283FE',
 'Chloroflexi': '#511CFB',
 'Cyanobacteria': '#00A08B',
 'Deinococcus-Thermus': '#3283FE',
 'Deltaproteobacteria': '#DAE8AA',
 'Euryarchaeota': '#FC0080',
 'Firmicutes': '#B2828D',
 'Gammaproteobacteria': '#98C995',
 'Gemmatimonadetes': '#6C7C32',
 'Halobacteria': '#3283FE',
 'Ignavibacteriae': '#DEA0FD',
 'Nitrososphaeria': '#FE00FA',
 'Nitrospinae': '#862A16',
 'Nitrospirae': '#A777F1',
 'Oligoflexia': '#FF8028',
 'Planctomycetes': '#620042',
 'Spirochaetes': '#F6222E',
 'Thaumarchaeota': '#1CFFCE',
 'Verrucomicrobia': '#2ED9FF',
 'Zetaproteobacteria': '#63680c',
 'candidate division Zixibacteria': '#C075A6',
 'Proteobacteria': '#3C6A54',


In [51]:
list( set(tax_class) ^ set(phyla2color.keys()) )

['Zetaproteobacteria',
 'Gemmatimonadetes',
 'Crenarchaeota',
 'Actinobacteria',
 'Cyanobacteria',
 'Nitrospinae',
 'Oligoflexia',
 'Firmicutes',
 'Proteobacteria',
 'Acidithiobacillia']

In [25]:
gene='nirS'

In [51]:
# to color strip

out_text_phylum = to_color_strip(acc2phylum,phyla2color,'phylum')

acc2proteocla = {k:v for k,v in acc2phyclas.items() 
                 if v not in acc2phylum.values()}

out_text_proteoclass = to_color_strip(acc2proteocla,
                                      phyclas2color,
                                      'Proteobacteria Archaeota class')

os.chdir('/share/home-user/hyc/work/5_tree/annotate')
with open('./colorstrip/%s_archaea_gene_colorstrip_phylum.txt' 
          %(gene),'w') as f:
    f.write(out_text_phylum)

with open('./colorstrip/%s_archaea_gene_colorstrip_protepclass.txt' 
          %(gene), 'w') as f:
    f.write(out_text_proteoclass)

In [54]:
# to sequence length
acc2len = {record.id:len(record.seq) for record in SeqIO.parse('/share/home-user/hyc/work/3_cdhit/ref_10575/cat_ref/'+gene+'_gene_ref.nr97.fasta','fasta') 
           if record.id in tree_acc}

out_text_bar = simplebar(acc2len,'%s gene sequence length' % gene)

os.chdir('/share/home-user/hyc/work/5_tree/annotate')
with open('./len/%s_gene_len.txt' 
          %(gene), 'w') as f:
    f.write(out_text_bar)

In [55]:
# to cluster size
os.chdir('/share/home-user/hyc/work/3_cdhit/ref_10575/')
text = ''
acc2clstr=dict()

for line in open(gene+'_criteria.nr97.fasta.clstr'):
    if re.match('^>Cluster',line):
        if text:
            clstr = text.split('\n')[:-1]
            for _ in clstr:
                if '*' in _:
                    accid = _.split('>')[-1].split('...')[0]
                    clstr_no = len(clstr)
                    acc2clstr[accid]=clstr_no
                    break
            text = ''
    else:
        text += line

out_text_bar = simplebar(acc2clstr,'%s gene clstr' % gene)

os.chdir('/share/home-user/hyc/work/5_tree/annotate')
with open('./len/%s_gene_clstr.txt' 
          %(gene), 'w') as f:
    f.write(out_text_bar)

In [56]:
# to color style
out_text_styles=to_color_style(acc2phyclas,phyla2color,'branch color','label,node',bg=True)

os.chdir('/share/home-user/hyc/work/5_tree/annotate')
with open('./colorstyle/%s_gene_colorstyle_phyclas.txt' 
          %(gene), 'w') as f:
    f.write(out_text_styles)

In [10]:
# to label text
out_text_label = label_text(acc2phylum)

os.chdir('/share/home-user/hyc/work/5_tree/annotate')
with open('./%s_gene_labeltext.txt' 
          %(gene), 'w') as f:
    f.write(out_text_label)

In [11]:
# to pop label
out_popup = to_popup(acc2phylum)

os.chdir('/share/home-user/hyc/work/5_tree/annotate')
with open('./%s_gene_popup_acc.txt' 
          %(gene), 'w') as f:
    f.write(out_popup)

In [59]:
# to branch color
out_branch_color = to_color_range(acc2phylum,phyla2color,pos='branch')

os.chdir('/share/home-user/hyc/work/5_tree/annotate')
with open('./colorrange/%s_gene_branch_color.txt' 
          %(gene), 'w') as f:
    f.write(out_branch_color)

In [73]:
# annote gene type from diamond out with color range
# -*- coding: UTF-8 -*-

gene = 'norB_ref'
filepath='/share/home-user/hyc/work/1_diamond/ref_10575/'+ gene +'.diamond.out'
os.chdir('/share/home-user/hyc/work/2_blast_extract/ref_10575/')


df = pd.read_csv(filepath, sep = '\t',header=None)
df.columns= ['query','subject_header','subject',
             'identity','length','e-value','score','qlen','slen']

# iden > 30
select_ident = df.loc[df['identity']>30]

# evalue < 1e-20
select_ev = select_ident.loc[select_ident['e-value']<1e-20]

# form the drop row index_list
drop_index = [int(x) for i,x in enumerate(select_ev.index) 
              if select_ev.iat[i,4]/select_ev.iat[i,7]<0.6 
#               or select_ev.iat[i,4]/ref_len[select_ev.iat[i,0]]<0.6 
              or select_ev.iat[i,4]/select_ev.iat[i,8]<0.6 ]

# do the drop
select_cov = select_ev.drop(index = drop_index)

os.chdir('/share/home-user/hyc/work/5_tree/')
gene = 'norB'
ref_ids = [_.id for _ in SeqIO.parse('/share/home-user/hyc/work/0_seq/ref/norB_ref.fasta','fasta')]
# og_ids = [_.id for _ in SeqIO.parse('/share/home-user/hyc/work/0_seq/outgroup/norB_outgroup.fasta','fasta')]


df = pd.read_excel('total_ref_outgroup.xlsx',
                   usecols=['fa_name','gene'])

df = df[df['fa_name'].isin(ref_ids)]

query2gene = dict(zip(df['fa_name'],df['gene']))
sub2query = list(zip(select_cov['subject_header'],select_cov['query']))

sub2gene = [(_[0],query2gene[_[1]]) for _ in sub2query]

uq_s2g={}
for _ in sub2gene:
    if _[0] in uq_s2g.keys():
        uq_s2g[_[0]].append(_[1])
    else:
        uq_s2g[_[0]] = [_[1]]

hit_twice = {k:v for k,v in uq_s2g.items() if len(list(set(v)))>1}

acc2gene = {k:list(set(v)) for k,v in uq_s2g.items()}

In [96]:
acc2gene = {k:'\t'.join(list(set(v))) for k,v in uq_s2g.items()}
gene2color['cnorB']='#82D5FF'
gene2color['qnorB']='#F8A618'
gene2color['cnorB\tqnorB']='#72B7B2'
gene2color['qnorB\tcnorB']='#72B7B2'

out_text_range = to_color_range(acc2gene,gene2color)
os.chdir('/share/home-user/hyc/work/5_tree/annotate')
with open('./colorrange/%s_gene_type_range.txt' %gene,'w') as f:
    f.write(out_text_range)

signalp -fasta ../ref_10575/cat_ref/nosZ_gene_ref.nr97.fasta  -org gram- -format short -prefix nosZ_short_gram_neg

In [54]:
# signal peptide annotate
df_sg = pd.read_csv('/share/home-user/hyc/work/3_cdhit/ref_10575/cat_ref/nosZ_short_gram_neg_summary.signalp5',sep='\t',header=1)

acc2signal = dict(zip(df_sg['# ID'],df_sg['Prediction']))

set(acc2signal.values())

from collections import Counter
cnt = Counter()
for _ in acc2signal.values():
    cnt[_] +=1
print(cnt)

# for k,v in acc2signal.items():
#     if k in ref_og_ids:
#         acc2signal[k] += ' ref'

signal2color ={
#     'TAT(Tat/SPI) ref':'#1B9E77',
#     'LIPO(Sec/SPII) ref':'#D95F02',
#     'SP(Sec/SPI) ref':'#D95F02',
#     'OTHER ref':'#7570B3',
    'TAT(Tat/SPI)':'#FDCDAC',
    'LIPO(Sec/SPII)':'#E6F5C9',
    'SP(Sec/SPI)':'#B3E2CD',
    'OTHER':'#CBD5E8'
}

out_text_range = to_color_range(acc2signal,signal2color)
os.chdir('/share/home-user/hyc/work/5_tree/annotate')
with open('./colorrange/%s_signal_type_range.txt' %gene,'w') as f:
    f.write(out_text_range)

Counter({'TAT(Tat/SPI)': 340, 'LIPO(Sec/SPII)': 257, 'SP(Sec/SPI)': 109, 'OTHER': 46})


In [72]:
def to_binary_ref(ref_ids,
                  sep = ',',
                  shape = '2',
                  field_labels = 'ref',
                  label = 'reference'):
    os.chdir('/share/home-user/hyc/db/itol_template')
    template_text = open('./dataset_binary_template.txt').read()
    template_text = template_text.format(shape=shape,
                                         field_labels=field_labels,
                                         dataset_label=label)
    annotate_text = '\n'.join(['%s%s1'%(_,sep) for _ in ref_ids])
    
    out_text = template_text + '\n' + annotate_text
    return out_text

In [74]:
out_ref_binary = to_binary_ref(ref_og_ids)
os.chdir('/share/home-user/hyc/work/5_tree/annotate')
with open('./binary/%s_ref_binary.txt' %gene,'w') as f:
    f.write(out_ref_binary)

In [67]:
gene = 'nirS'
filepath='/share/home-user/hyc/work/1_diamond/ref_10575/'+ gene +'.diamond.out'

df = pd.read_csv(filepath, sep = '\t',header=None)
df.columns= ['query','subject_header','subject',
             'identity','length','e-value','score','qlen','slen']

# iden > 30
select_ident = df.loc[df['identity']>30]

# evalue < 1e-20
select_ev = select_ident.loc[select_ident['e-value']<1e-20]

# form the drop row index_list
drop_index = [int(x) for i,x in enumerate(select_ev.index) 
              if select_ev.iat[i,4]/select_ev.iat[i,7]<0.6 
#               or select_ev.iat[i,4]/ref_len[select_ev.iat[i,0]]<0.6 
              or select_ev.iat[i,4]/select_ev.iat[i,8]<0.6 ]

# do the drop
select_cov = select_ev.drop(index = drop_index)


In [68]:
sub2query = list(zip(select_cov['subject_header'],select_cov['query']))

In [69]:
uq_s2q={}
for _ in sub2query:
    if _[0] in uq_s2q.keys():
        uq_s2q[_[0]].append(_[1])
    else:
        uq_s2q[_[0]] = [_[1]]

In [74]:
os.chdir('/share/home-user/hyc/work/5_tree/')
gene = 'nirS'
ref_ids = [_.id for _ in SeqIO.parse('/share/home-user/hyc/work/0_seq/ref/%s_ref.fasta'%gene,'fasta')]
# og_ids = [_.id for _ in SeqIO.parse('/share/home-user/hyc/work/0_seq/outgroup/norB_outgroup.fasta','fasta')]

df = pd.read_excel('total_ref_outgroup.xlsx')

In [78]:
df[df['accession'].isin(uq_s2q['NZ_JRKO01000011.1_53'])]

,type,gene name,accession,GI,gene,organ,seq,tax,phylum,reference,version,taxid,len,fa_name
142,ref,nirS,CCK76416,508730931,nirS,Oleispira antarctica RB-8,MKKLKFGLSVIGLAIGMATAGVSTAAPAKGPTLSEKVFEETSLLYF...,Bacteria_Proteobacteria_Gammaproteobacteria_Oc...,Proteobacteria,Genome sequence and functional genomic analysi...,CCK76416.1,698738,543,gi|508730931|emb|CCK76416.1|[508730931]
